# Klassifikation mit BERT

Du hast nun bereits mit den BERT-Embeddings gearbeitet. Die eigentliche Leistungsfähigkeit der Sprachmodelle steckt allerdings in dem sog. *Finetuning*, d.h. sie speziell auf deine Anforderungen anzupassen.

In diese Teil wirst du kennenlernen, wie sich das auf die Klassifikation der Heise-Artikel nach solchen mit vielen oder wenigen Kommentaren anwenden lässt.

## Daten einladen

BERT enthält selbst einen Tokenisierer, daher verwendest du direkt die Titel:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, datePublished, title, commentCount FROM articles \
                    WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Daten für Klassifikation vorbereiten

Zunächst normalisierst du die Kommentare:

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

Dann konstruierst du zwei `DataFrame`, in denen erfolgreich und nicht erfolgreiche Posts enthalten sind:

In [ ]:
df_success = df[df["normalizedCommentCount"]>50].copy()
df_success["success"] = 1

df_no_success = df[df["normalizedCommentCount"]<10].copy()
df_no_success["success"] = 0

Du berechnest die Größe des kleineren `DataFrame`:

In [ ]:
min_success = min(len(df_success), len(df_no_success))

Und erzeugst ein ausgeglichenes Trainingsset:

In [ ]:
sdf = pd.concat([df_success.sample(min_success, random_state=42),
                 df_no_success.sample(min_success, random_state=42)])

## Transfer Learning

Für diesen Teil musst du sehr umfangreiche Berchnungen durchführen. Diese funktionieren zwar grundsätzlich auch auf einer CPU, allerdings würde das viele Stunden dauern. 

Wenn möglichst, solltest du den Code daher auf einer Grafikkarte laufen lassen, auf der das viele Größenordnungen schneller funktioniert. Solltest du auf keine Grafikkarte zugreifen können, lohnt es sich, dieses Noteboook in Google Colab auszuführen und eine entsprechende Umgebung auszuwählen.

In [ ]:
!pip install torch

In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print("Using GPU %s" % torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU :-(")


Die `GeForce RTX 2070` ist zwar nicht ganz modern, aber für unsere Zwecke ist sie absolut ausreichend.

Aus der `transformers`-Bibliothen nutzt du zunächst den `BertTokenizer`, um die Titel in ihre Bestandteil zu zerlegen:

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

BERT bzw. die Bibliotheken können nicht so elegant wie `scikit-learn` mit `DataFrame`s umgehen, daher musst du die Daten in Arrays konvertieren:

In [ ]:
text = sdf["title"].values
labels = sdf["success"].values

In vielen Layern benötigt das BERT-Modell Platz, der proportional zur Maximallänge der Text ist. Um hier zu sparen, bestimmst du zunächst maximale Länge:

In [ ]:
max_len = max([len(tokenizer.encode(t, add_special_tokens=True)) for t in text])
max_len

Jetzt bestimmst du die *Input IDs* und die *Attention Masks*:

In [ ]:
input_ids = []
attention_masks = []

for t in text:
    encoded_dict = tokenizer.encode_plus(
                        t,
                        add_special_tokens = True,    # '[CLS]' und '[SEP]'
                        max_length = 64,
                        truncation = True,
                        padding='max_length',
                        return_attention_mask = True,  # Attention-Masks erzeugen
                        return_tensors = 'pt',         # pytorch-Tensoren als Ergebnis
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

Aus technischen Gründen musst du die oben erzeugten Listen jetzt in *Tensoren* wandeln, mit denen `PyTorch` als Basisobjekte arbeitet:

In [ ]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Nun kannst du dir anzeigen lassen, was der `BertTokenizer` aus deinem ersten Dokument gemacht hat:

In [ ]:
print(text[0])
print(tokenizer.tokenize(text[0]))
print(input_ids[0])

Wie du siehst, hat der Tokenizer auch Subworte erkannt. Nur diese Worte kennt das dazugehörige Modell auch, deshalb ist es absolut entscheidend, immer den passenden Tokenizer mit dem dazueghörigen Modell zu verwenden!

Nun erzeugst du ein Datenset, das - du ahnst es bereits - auch wieder im Tensor-Format vorliegen muss.

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

Ähnlich wie bei `scikit-learn` gibt es auch hier eine Hilfsfunktion, mit der du das Datenset in Trainings- und Testdaten aufteilst:

In [ ]:
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
# reproduzierbar arbeiten!
torch.manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(train_size, val_size)

Anders als bei `scikit-learn` arbeitet das Training in sog. *Batches*, deren Größe du hier mit 32 festlegst, wie der BERT-Autoren das empfehlen:

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

Anschließend erzeugst du sog. `DataLoader`, die dir die Daten für die Batches genau so bereitstellen, wie du sie jeweils brauchst:

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

Nun instanziierst du das Modell, dort musst du den gleichen Namen wie beim `BertTokenizer` oben verwenden. Wenn du keine Grafikkarte hast, musst du dich auf sehr lange Wartezeiten einstellen und `model.cuda()` durch `model.cpu()` ersetzen.

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# das Modell muss zum Tokenizer passen!
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2, # wir haben nur gut oder shlecht
    output_attentions = False,
    output_hidden_states = False # wir benötigen keine Embeddings
)
# hier evtl. model.cpu() einsetzen
model.cuda()

Das Modell versucht anschließend, den sog *Loss* (also die fehlerhaft klassifizierten Daten) zu minimieren. Dazu gibt es verschiedene Strategien, `AdamW` ist eigentlich die gebräuchliste:

In [ ]:
# Optimierer auswählen, AdamW ist Standard
optimizer = AdamW(model.parameters(), lr = 2e-5)

Das Modell wird in sog. `Epochen` trainiert, also einfach mehrmals hintereinander. Für unser Beispiel wählst du vier Epochen und erzeugst zusammen mit den Batches und der Größe des Trainingsets einen darauf angepassten *Scheduler*.

In [ ]:
from transformers import get_linear_schedule_with_warmup

# vier Epochen, das muss evtl. justiert werden
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

Du benötigst nun noch eine Funktion, die dir die Accuracy berechnet (durch das ausgeglichene Datenset ist das hier völlig in Ordnung). Die ist ein bisschen komplizierter. Von den Vorhersagen nutzt du nur die mit der jeweils größten Wahrscheinlichkeit für das Label. Da die Accuracy immer für einen ganzen *Batch* ausgerechnet werden muss, musst du die richtigen Vorhersagen zusammenzählen und durch die Anzahl der Datensätze teilen (`len(labels_flat)`):

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Jetzt kannst du mit den eigentlichen Training beginnnen. Da die Funktion relativ kompliziert ist, werden wir sie im nächsten Teil genauer besprechen.

Bitte brich hier das Notebook nicht ab, sondern fahre in der nächsten Lektion genau an dieser stelle fort.

Wunderbar! Genau hier!

Wir schauen uns jetzt gemeinsam die Funktion zum Training an.

In [ ]:
%%time
import random
import numpy as np
from tqdm.auto import trange, tqdm

# alle Zufallszahlengeneratoren initialisieren (Reproduzierbarkeit)
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Statistik für das Training
training_stats = []

for epoch_i in trange(epochs, desc="Epoche"):
    # akkumulierter Loss für diese Epoche
    total_train_loss = 0

    # Modell in Trainingsmodus stellen
    model.train()

    # Batchweise trainieren
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        # Daten entpacken und in device-Format wandeln
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Gradienten löschen
        model.zero_grad()        

        # Vorwärts-Auswertung (Trainingsdaten vorhersagen)
        res = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Loss berechnen und akkumulieren
        total_train_loss += res.loss.item()

        # Rückwärts-Auswertung, um Gradienten zu bestimmen
        res.loss.backward()

        # Gradient beschränken wegen Exploding Gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Parameter und Lernrate aktualisieren
        optimizer.step()
        scheduler.step()

    # Mittleren Loss über alle Batches berechnen
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
      

    # Modell in Vorhersage-Modus umstellen
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, desc="Validierung"):
        # jetzt die Validierungs-Daten entpacken
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Rückwärts-Auswertung wird nicht benötigt, daher auch kein Gradient
        with torch.no_grad():        
            # Vorhersage durchführen
            res = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Loss akkumulieren
        total_eval_loss += res.loss.item()

        # Vorhersagedaten in CPU-Format wandeln, um Accuracy berechnen zu können
        logits = res.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Gesamt-Accuraccy für diese Validierung ausgeben.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    tqdm.write("Accuracy: %f" % avg_val_accuracy)

    # Gesamte Loss über alle batches berechnen
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    tqdm.write("Validation loss %f" % avg_val_loss)

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validierung Loss': avg_val_loss,
            'Accuracy': avg_val_accuracy
        }
    )

Die Ergebnisse (d.h. den Loss und die Accuracy) kannst du am besten in einem `DataFrame` darstellen:

In [ ]:
import pandas as pd

df_stats = pd.DataFrame(data=training_stats).set_index("epoch")
df_stats

Lass dir das auch visualisieren:

In [ ]:
df_stats.plot()

Man könnte sich vorstellen, dass für das Training auch drei Epochen genügt hätten. BERT-Modelle haben leider auch eine gewisse Tendenz zum *Overfitting*.

## Finetuning als Basis für eigene Aufgaben

BERT ist besonders leistungsfähig, weil du es auf deine eigenen Bedürfnisse anpassen kannst. Dazu benötigst du das *Finetuning*, das du gerade kennengelernt hast.

In diesem Szenario hat das Training nicht sehr lange gedauert. Je nach Datenmenge und auch abhängig von deiner Hardware-Ausstattung kann das bei dir ganz anders aussehen. Im Vergleich zu den einfachen Machine-Learning-Modellen brauchen diese Deep-Learning-Methoden jedenfalls fast immer sehr viel mehr Rechenzeit.